In [12]:
# pc 2/20/2026
import requests
import pandas as pd
import dbf

# ==============================
# CONFIG
# ==============================
ACS_KEY = "your key"
ACS_URL = "https://api.census.gov/data/2024/acs/acs5"
DEC_URL = "https://api.census.gov/data/2020/dec/pl"

ACS_VARS = "B01001_001E,B11001_001E,B23025_003E,NAME"
DEC_VARS = "P5_001N,P5_002N,P5_003N"

COUNTIES = [
    ("San Francisco", "075"),
    ("San Mateo", "081"),
    ("Santa Clara", "085"),
    ("Alameda", "001"),
    ("Contra Costa", "013"),
    ("Solano", "095"),
    ("Napa", "055"),
    ("Sonoma", "097"),
    ("Marin", "041"),
    ("Santa Cruz", "087"),
    ("Monterey", "053"),
    ("San Benito", "069"),
    ("San Joaquin", "077")
]

# ==============================
# STEP 0: LOAD COUNTY CONTROL TOTALS
# ==============================
acs5_county_file = "acs5_2024_13counties_population_county.csv"
acs5_county = pd.read_csv(acs5_county_file)
acs5_county.rename(columns={
    "County_Name": "COUNTY",
    "Population_in_Households": "HH_POP_CNTY",
    "Total_Households": "TOT_HH_CNTY",
    "Total_Population": "TOT_POP_CNTY"
}, inplace=True)

# ==============================
# STEP 1: GET DECENNIAL 2020 GQ LOCATIONS
# ==============================
dec_all = []

for cname, cfips in COUNTIES:
    print(f"Decennial BG GQ: {cname}")
    params = {
        "get": DEC_VARS,
        "for": "block group:*",
        "in": f"state:06 county:{cfips}"
    }
    r = requests.get(DEC_URL, params=params)
    r.raise_for_status()
    data = r.json()

    df = pd.DataFrame(data[1:], columns=data[0])
    df["P5_003N"] = pd.to_numeric(df["P5_003N"], errors="coerce").fillna(0)

    df["state"] = df["state"].str.zfill(2)
    df["county"] = df["county"].str.zfill(3)
    df["tract"] = df["tract"].str.zfill(6)
    df["block group"] = df["block group"].str.zfill(1)

    df["GEOID"] = df["state"] + df["county"] + df["tract"] + df["block group"]
    df["COUNTY"] = cname
    df["GRP_QUA"] = df["P5_003N"]  # KEEP EXACT 2020 GQ

    # Keep only BGs with GQ > 0
    df = df[df["GRP_QUA"] > 0]

    dec_all.append(df[["GEOID", "COUNTY", "GRP_QUA", "state", "county", "tract", "block group"]])

dec_bg = pd.concat(dec_all, ignore_index=True)

# ==============================
# STEP 2: GET ACS 2024 BG DATA
# ==============================
acs_all = []

for cname, cfips in COUNTIES:
    print(f"ACS BG totals: {cname}")
    params = {
        "get": ACS_VARS,
        "for": "block group:*",
        "in": f"state:06 county:{cfips}",
        "key": ACS_KEY
    }
    r = requests.get(ACS_URL, params=params)
    r.raise_for_status()
    data = r.json()

    df = pd.DataFrame(data[1:], columns=data[0])

    for c in ["B01001_001E", "B11001_001E", "B23025_003E"]:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

    df.rename(columns={
        "B01001_001E": "TOT_POP",
        "B11001_001E": "TOT_HH",
        "B23025_003E": "EMP_RES",
        "block group": "BLKGRP"
    }, inplace=True)

    df["state"] = df["state"].str.zfill(2)
    df["county"] = df["county"].str.zfill(3)
    df["tract"] = df["tract"].str.zfill(6)
    df["BLKGRP"] = df["BLKGRP"].str.zfill(1)
    df["GEOID"] = df["state"] + df["county"] + df["tract"] + df["BLKGRP"]
    df["COUNTY"] = cname

    acs_all.append(df)

acs_bg = pd.concat(acs_all, ignore_index=True)

# ==============================
# STEP 3: MERGE GQ INTO ACS BG
# ==============================
final_df = acs_bg.copy()
final_df = final_df.merge(dec_bg[["GEOID","GRP_QUA"]], on="GEOID", how="left")
final_df["GRP_QUA"] = final_df["GRP_QUA"].fillna(0)

# ==============================
# STEP 4: COMPUTE HH_POP safely
# ==============================
# Prevent negative HH_POP
final_df["GRP_QUA"] = final_df[["GRP_QUA","TOT_POP"]].min(axis=1)
final_df["HH_POP"] = final_df["TOT_POP"] - final_df["GRP_QUA"]

# Scale HH_POP to match ACS county totals exactly
for cname in final_df["COUNTY"].unique():
    county_mask = final_df["COUNTY"] == cname
    county_hh_pop = final_df.loc[county_mask, "HH_POP"].sum()
    target_hh_pop = acs5_county.loc[acs5_county["COUNTY"]==cname, "HH_POP_CNTY"].values[0]
    if county_hh_pop > 0:
        scale = target_hh_pop / county_hh_pop
        final_df.loc[county_mask, "HH_POP"] = (final_df.loc[county_mask, "HH_POP"] * scale).round().astype(int)

# ==============================
# STEP 5: WRITE DBF
# ==============================
final_df["GEOID_TX"] = final_df["GEOID"].astype(str)
final_df["GEOID_JN"] = "0" + final_df["GEOID_TX"]

dbf_file = "acs2024_bg_with_gq_allocated_scaled.dbf"

table = dbf.Table(
    dbf_file,
    """
    GEOID       C(12);
    GEO_TX      C(12);
    GEO_JN      C(13);
    COUNTY      C(20);
    TOT_HH      N(10,0);
    HH_POP      N(10,0);
    TOT_POP     N(10,0);
    EMP_RES     N(10,0);
    GRP_QUA     N(10,0);
    GQ_TRUNC    N(1,0);
    STATE       C(2);
    CNTY        C(3);
    TRACT       C(6);
    BLKGRP      C(1);
    NAME        C(60)
    """
)
table.open(mode=dbf.READ_WRITE)

for _, row in final_df.iterrows():
    table.append((
        row["GEOID"],
        row["GEOID_TX"],
        row["GEOID_JN"],
        row["COUNTY"],
        int(row["TOT_HH"]),
        int(row["HH_POP"]),
        int(row["TOT_POP"]),
        int(row["EMP_RES"]),
        int(row["GRP_QUA"]),
        int((row["GRP_QUA"] > row["TOT_POP"])),  # GQ_TRUNC
        row["state"],
        row["county"],
        row["tract"],
        row["BLKGRP"],
        row.get("NAME","")[:60]
    ))

table.close()
print("✅ DONE — HH_POP non-negative, county totals match ACS, GQ only at 2020 BGs")


Decennial BG GQ: San Francisco
Decennial BG GQ: San Mateo
Decennial BG GQ: Santa Clara
Decennial BG GQ: Alameda
Decennial BG GQ: Contra Costa
Decennial BG GQ: Solano
Decennial BG GQ: Napa
Decennial BG GQ: Sonoma
Decennial BG GQ: Marin
Decennial BG GQ: Santa Cruz
Decennial BG GQ: Monterey
Decennial BG GQ: San Benito
Decennial BG GQ: San Joaquin
ACS BG totals: San Francisco
ACS BG totals: San Mateo
ACS BG totals: Santa Clara
ACS BG totals: Alameda
ACS BG totals: Contra Costa
ACS BG totals: Solano
ACS BG totals: Napa
ACS BG totals: Sonoma
ACS BG totals: Marin
ACS BG totals: Santa Cruz
ACS BG totals: Monterey
ACS BG totals: San Benito
ACS BG totals: San Joaquin
✅ DONE — HH_POP non-negative, county totals match ACS, GQ only at 2020 BGs
